PROYECTO ETL

In [1]:
import requests as req
import os
from dotenv import load_dotenv
import base64
from requests import post,get
import json
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
load_dotenv()
client_id=os.getenv('CLIENT_ID')
client_secret=os.getenv('CLIENT_SECRET')

In [3]:
#para entrar a la api
def get_token():
    auth_string = client_id + ':' + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = base64.b64encode(auth_bytes).decode('utf-8')

    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': 'Basic ' + auth_base64,
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    data = {'grant_type': 'client_credentials'}
    result = post(url, headers=headers, data=data)
    
    json_result = result.json()
    token = json_result.get('access_token', None) 
    return token

token = get_token()


In [4]:
def get_auth_header(token):
    return {'Authorization':'Bearer '+ token}


In [6]:
#Funcion para buscar unicamente los id de los albums
def search_for_album_id(token, album_name):
    url = 'https://api.spotify.com/v1/search'
    headers = {'Authorization': f'Bearer {token}'}
    params = {
        'q': album_name,
        'type': 'album',
        'limit': 1
    }

    response = req.get(url, headers=headers, params=params)
    json_result = response.json()

    # Buscar el ID del álbum en los resultados y devolverlo
    albums_items = json_result.get('albums', {}).get('items', [])
    if albums_items:

        return albums_items[0].get('id')
    else:
        return None

# Importo el database que limpie para sacar los albums id

In [7]:
#Creo una nueva tabla para las canciones de los albums
canciones = pd.read_csv('../database/csv_limpios/albums.csv')

In [8]:
canciones['album_id'] = canciones['Album'].apply(lambda x: search_for_album_id(token, x))

In [9]:
canciones = canciones.drop(columns=['Year', 'Ranking','Worldwide Sales','CDs','Genre','Tracks',])


In [10]:
lista_id = canciones['album_id'].tolist()

In [11]:
# Funcion para obtener las canciones de los albums
def cancion(album_id, token):
    headers = {'Authorization': f'Bearer {token}'}
    url = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
    try:
        response = req.get(url, headers=headers)
        album_tracks = response.json().get('items', []) 
        return [track['name'] for track in album_tracks]  
    except:
        pass

In [12]:
#Aplico la funcion para todos los albums_id
canciones['canciones'] = canciones['album_id'].apply(lambda x: cancion(x, token))

In [13]:
#Aplico pd.Series para separar en cada columna una cancion
canciones2 = canciones['canciones'].apply(pd.Series)

In [14]:
#uno los dataframes a solo uno
canciones = pd.concat([canciones, canciones2], axis=1).drop('canciones', axis=1)

In [15]:
#Renombro las columnas
canciones = canciones.rename(columns={
                                        0: 'Cancion 1', 1: 'Cancion 2', 2: 'Cancion 3', 3: 'Cancion 4', 4: 'Cancion 5',
                                        5: 'Cancion 6', 6: 'Cancion 7', 7: 'Cancion 8', 8: 'Cancion 9',
                                        9: 'Cancion 10', 10: 'Cancion 11', 11: 'Cancion 12', 12: 'Cancion 13',
                                        13: 'Cancion 14', 14: 'Cancion 15', 15: 'Cancion 16', 16: 'Cancion 17'
                                    })

In [16]:
#Cambio los nan
canciones.fillna('no tiene', inplace=True)

# Agrego una columna de popularidad a la tabla de albums usando la api


In [17]:
albums2 = pd.read_csv('../database/csv_limpios/albums.csv')

In [18]:
#Primero agrego los album_id a la tabla
albums2['album_id'] = albums2['Album'].apply(lambda x: search_for_album_id(token, x))

In [19]:
# Popularidad de los albums del ranking hoy en dia, esta entre 0 y 100 siendo 100 el mas popular
#La popularidad se calcula por la cantidad de reproducciones de las pistas del álbum y qué tan recientes son esas reproducciones.
def popularity(token, album_id):
    
    url = f'https://api.spotify.com/v1/albums/{album_id}'
    headers = {'Authorization': f'Bearer {token}'}

    response = req.get(url, headers=headers)
    album_details = response.json()

    popularity = album_details.get('popularity')

    return popularity

albums2['popularity'] = albums2['album_id'].apply(lambda x: popularity(token, x))

In [20]:
albums2

,Year,Ranking,Artist,Album,Worldwide Sales,CDs,Tracks,Genre,album_id,popularity
0,1990,1,Madonna,The Immaculate Collection,"30,000,000",1,17,Pop,5MzwGwnO9gkh0z6Nl4FF8h,61
1,1990,2,New_Kids_on_the_Block,Step By Step,"20,000,000",1,12,Pop,4dCdnfD0shXGuHtyTOtx8q,54
2,1990,3,Garth Brooks,No Fences,"18,770,000",1,10,Country,2l8LCtgshAUZlSZbLAzWr4,4
3,1991,1,Metallica,Metallica,"33,000,000",1,12,Rock,55fq75UfkYbGMq4CncCtOH,72
4,1991,2,Nirvana_(banda),Nevermind,"30,000,000",1,13,Rock,2guirTSEqLizK7j9i1MTTZ,77
...,...,...,...,...,...,...,...,...,...,...
91,2020,2,BTS,Be,"5,397,015",1,8,Pop,0P3oVJBFOv3TDXlYRhGL7s,85
92,2020,3,Pop Smoke,"Shoot For The Stars, Aim For The Moon","4,777,389",1,19,Hip Hop,7e7t0MCrNDcJZsPwUKjmOc,81
93,2021,1,Adele,30,"4,485,025",1,12,Pop,21jF5jlMtzo94wbxmJ18aa,80
94,2021,2,Olivia Rodrigo,Sour,"3,830,500",1,11,Pop,6s84u2TUpR3wdUv4NgKA2j,90


# Exporto canciones 

In [21]:
canciones.to_csv('../database/csv_limpios/canciones.csv', index=False)

albums2.to_csv('../database//csv_limpios/albums_completo.csv', index=False)